# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df=pd.read_csv("output_data/cities.csv")

weather_df


,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Cloudiness,Wind Speed,Humidity,Date
0,0,Ribeira Grande,PT,38.5167,-28.7000,61.20,40,16.11,82,1643993085
1,1,Ulaangom,MN,49.9811,92.0667,-21.33,80,3.13,93,1643993085
2,2,Saskylakh,RU,71.9167,114.0833,-7.92,100,4.25,99,1643993039
3,3,Jamestown,US,42.0970,-79.2353,21.09,100,5.01,62,1643993085
4,4,Saint George,US,37.1041,-113.5841,41.47,0,1.01,44,1643992861
...,...,...,...,...,...,...,...,...,...,...
517,517,Uíge,AO,-7.6087,15.0613,71.42,96,2.73,87,1643993829
518,518,Anáhuac,MX,27.2333,-100.1500,34.54,100,22.01,38,1643993829
519,519,Darovskoy,RU,58.7700,47.9564,14.29,100,7.09,94,1643993829
520,520,Coahuayana Viejo,MX,18.7333,-103.6833,81.88,4,3.44,30,1643993830


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Create variables for heatmap
lat_lng_pairs = weather_df[['Latitude', 'Longitude']]
weights = weather_df['Humidity']
#Generate Heatmap using gmaps
gfig = gmaps.figure()
gfig.add_layer(gmaps.heatmap_layer(lat_lng_pairs, weights=weights, 
                                   max_intensity=weather_df['Humidity'].max().astype('float')))
gfig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
#Removing rows that don't match conditions

hotel_df=weather_df[(weather_df['Max Temperature']>70)&(weather_df['Max Temperature']<80)&\
                  (weather_df["Wind Speed"]<10)&(weather_df['Cloudiness']==0)].dropna().reset_index()

hotel_df

,index,Unnamed: 0,City,Country,Latitude,Longitude,Max Temperature,Cloudiness,Wind Speed,Humidity,Date
0,42,42,Busselton,AU,-33.6500,115.3333,72.09,0,5.41,67,1643993096
1,49,49,Cape Town,ZA,-33.9258,18.4232,72.10,0,5.99,49,1643992934
2,115,115,Geraldton,AU,-28.7667,114.6000,72.01,0,3.44,60,1643993236
3,122,122,Cabo San Lucas,MX,22.8909,-109.9124,70.52,0,1.41,53,1643993238
4,271,271,Salalah,OM,17.0151,54.0924,72.52,0,1.34,65,1643993459
5,328,328,Medina,SA,24.4686,39.6142,70.23,0,9.22,13,1643993536
6,379,379,São Filipe,CV,14.8961,-24.4956,74.82,0,7.74,68,1643993610
7,441,441,Sharjah city,AE,25.3573,55.4033,73.38,0,6.91,60,1643993397
8,506,506,Pisco,PE,-13.7000,-76.2167,74.01,0,8.05,73,1643993825


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Add Hotel Name to df
hotel_df = pd.DataFrame(hotel_df, columns =['City','Country','Latitude','Longitude','Max Temperature',
                                            'Cloudiness', 'Wind Speed', 'Humidity', 'Date', 'Hotel Name'])
hotel_df


,City,Country,Latitude,Longitude,Max Temperature,Cloudiness,Wind Speed,Humidity,Date,Hotel Name
0,Busselton,AU,-33.6500,115.3333,72.09,0,5.41,67,1643993096,NaN
1,Cape Town,ZA,-33.9258,18.4232,72.10,0,5.99,49,1643992934,NaN
2,Geraldton,AU,-28.7667,114.6000,72.01,0,3.44,60,1643993236,NaN
3,Cabo San Lucas,MX,22.8909,-109.9124,70.52,0,1.41,53,1643993238,NaN
4,Salalah,OM,17.0151,54.0924,72.52,0,1.34,65,1643993459,NaN
5,Medina,SA,24.4686,39.6142,70.23,0,9.22,13,1643993536,NaN
6,São Filipe,CV,14.8961,-24.4956,74.82,0,7.74,68,1643993610,NaN
7,Sharjah city,AE,25.3573,55.4033,73.38,0,6.91,60,1643993397,NaN
8,Pisco,PE,-13.7000,-76.2167,74.01,0,8.05,73,1643993825,NaN


In [ ]:
#Set up API calls
g_url="https://maps.googleapis.com/maps/api/place/details/json?"


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
